In [1]:
import scipy.io
import numpy as np
import networkx as nx
import os
from scipy.stats import ttest_rel, ttest_ind
import time   #预估运行时间(可删除)

In [2]:
# 储存文件路径
folder_path = '/Users/shenxiaoyu/Desktop/FC'

file_paths = [
    'shamfc_s1.mat', 'shamfc_s2.mat', 'shamfc_s3.mat',
    'tacs6fc_s1.mat', 'tacs6fc_s2.mat', 'tacs6fc_s3.mat',
    'tacs10fc_s1.mat', 'tacs10fc_s2.mat', 'tacs10fc_s3.mat',
    'tdcsfc_s1.mat', 'tdcsfc_s2.mat', 'tdcsfc_s3.mat'
]


# preprocessing brain data
def data_preprocessing(matrix, fc_threshold=0.25, zfc_threshold=1.5):
    # matrix = np.where(np.isinf(matrix), np.nan, matrix)  # Replace infinite values with NaN
    
    # Apply threshold to create a binary adjacency matrix
    threshold = zfc_threshold if 'zfc' in file_path.lower() else fc_threshold
    binary_weighted_matrix = np.where(np.abs(matrix) >= threshold, matrix, 0)
    
    # Remove self-loops
    np.fill_diagonal(binary_weighted_matrix, 0)
    
    return binary_weighted_matrix

# 创建一个dict来存储每种条件下的graph
graphs = {
    'sham': {'s1': [], 's2': [], 's3': []},
    'tacs6': {'s1': [], 's2': [], 's3': []},
    'tacs10': {'s1': [], 's2': [], 's3': []},
    'tdcs': {'s1': [], 's2': [], 's3': []}
}

# 读取文件里的matrix和提取ZFC
for file_name in file_paths:
    file_path = os.path.join(folder_path, file_name)
    mat = scipy.io.loadmat(file_path)
    key = [k for k in mat.keys() if not k.startswith('__')][0]  # 提取主要数据的key
    matrix = mat[key]
    
    # condition是刺激类型, time_point对应刺激前中后
    base_name = file_name.replace('.mat', '')
    parts = base_name.split('_')
    if len(parts) == 2:
        condition_time, time_point = parts
        if 'zfc' in condition_time:
            condition = condition_time.replace('zfc', '')
            matrix_type = 'zfc'
        else:
            condition = condition_time.replace('fc', '')
            matrix_type = 'fc'
    else:
        condition, matrix_type, time_point = parts

    
    if matrix_type == 'fc':
        # 将其转换为graph
        for i in range(matrix.shape[0]):
            adj_matrix_binary = data_preprocessing(matrix[i])
            G = nx.from_numpy_matrix(adj_matrix_binary)
            graphs[condition][time_point].append(G)   

In [3]:
# WhitakerLab. (n.d.). Scona: Structural Covariance Networks. Retrieved [date], from https://whitakerlab.github.io/scona/_modules/scona/graph_measures.html
def participation_coefficient(G, module_partition):
    """ Calculate the participation coefficient for each node in the graph. """
    pc_dict = {}
    for m in module_partition.keys():
        M = set(module_partition[m])
        for v in M:
            degree = float(nx.degree(G=G, nbunch=v))
            wm_degree = float(sum([1 for u in M if (u, v) in G.edges()]))
            pc_dict[v] = 1 - ((float(wm_degree) / float(degree)) ** 2)
    return pc_dict

In [4]:
# 定义所需要计算的graph property
def compute_graph_metrics(G):
    return {
        'degree_centrality': np.mean(list(nx.degree_centrality(G).values())),
        'betweenness_centrality': np.mean(list(nx.betweenness_centrality(G, weight='weight').values())),
        'eigenvector_centrality': np.mean(list(nx.eigenvector_centrality(G, max_iter=10000, weight='weight').values())),
        'clustering_coefficient': nx.average_clustering(G, weight='weight'),
        'modularity': nx.algorithms.community.quality.modularity(G, nx.community.greedy_modularity_communities(G, weight='weight'), weight='weight'),
        'connected_components': nx.number_connected_components(G),
        'minimum_spanning_tree': nx.minimum_spanning_tree(G, weight='weight', ignore_nan=True).size(weight='weight') if nx.is_connected(G) else None
        'rich_club_coefficient': 

    }

# 创建一个dict来储存计算结果
metrics = {
    'sham': {'s1': [], 's2': [], 's3': []},
    'tacs6': {'s1': [], 's2': [], 's3': []},
    'tacs10': {'s1': [], 's2': [], 's3': []},
    'tdcs': {'s1': [], 's2': [], 's3': []}
}

# 预估运行时间(可删除)
metrics_start_time = time.time()
total_graphs = sum(len(graphs[condition][time_point]) for condition in graphs for time_point in graphs[condition])
processed_graphs = 0

for condition in graphs:
    for time_point in graphs[condition]:
        for G in graphs[condition][time_point]:
            metrics[condition][time_point].append(compute_graph_metrics(G))
            
            
            # 预估运行时间(可删除)
            processed_graphs += 1
            elapsed_time = time.time() - metrics_start_time
            estimated_total_time = (elapsed_time / processed_graphs) * total_graphs
            remaining_time = estimated_total_time - elapsed_time
            print(f"Processed {processed_graphs}/{total_graphs} graphs. Estimated remaining time: {remaining_time:.2f} seconds.")

AttributeError: module 'networkx.algorithms.community.quality' has no attribute 'participation_coefficient'

In [ ]:
def compare_analysis(metrics, condition1, time_point1, condition2, time_point2, graph_property, paired=False):
    data1 = [m[graph_property] for m in metrics[condition1][time_point1] if m[graph_property] is not None]
    data2 = [m[graph_property] for m in metrics[condition2][time_point2] if m[graph_property] is not None]
    if len(data1) > 0 and len(data2) > 0:
        if paired:
            t_stat, p_value = ttest_rel(data1, data2)
        else:
            t_stat, p_value = ttest_ind(data1, data2)
    else:
        t_stat, p_value = None, None
    return t_stat, p_value

In [ ]:
# 1. 比较sham组在s1,s2,s3的差异 (期待没有显著差异)
"""print("Sham Group Stability:")
for graph_property in ['degree_centrality', 'betweenness_centrality', 'eigenvector_centrality', 'clustering_coefficient', 'modularity', 'rich_club_coefficient_new', 'connected_components', 'minimum_spanning_tree']:
    t_stat, p_value = compare_analysis(metrics, 'sham', 's1', 'sham', 's2', graph_property, paired=True)
    print(f"Sham S1 vs S2 - {graph_property}: T-statistic: {t_stat}, P-value: {p_value}")
    
    t_stat, p_value = compare_analysis(metrics, 'sham', 's1', 'sham', 's3', graph_property, paired=True)
    print(f"Sham S1 vs S3 - {graph_property}: T-statistic: {t_stat}, P-value: {p_value}")"""

In [ ]:
# 2. 验证四组s1相似性 (期待四组s1没有显著差异)
print("S_1:")
for condition in ['tacs6', 'tacs10', 'tdcs']:
    for graph_property in ['degree_centrality', 'betweenness_centrality', 'eigenvector_centrality', 'clustering_coefficient', 'modularity', 'rich_club_coefficient_new', 'participation_coefficient_new', 'connected_components', 'minimum_spanning_tree']:
        t_stat, p_value = compare_analysis(metrics, 'sham', 's1', condition, 's1', graph_property)
        print(f"Sham S1 vs {condition} S1 - {graph_property}: T-statistic: {t_stat}, P-value: {p_value}")

In [ ]:
# 3. 验证真实刺激的s2,s3与sham组的对比
print("S_2:")
for condition in ['tacs6', 'tacs10', 'tdcs']:
    for graph_property in ['degree_centrality', 'betweenness_centrality', 'eigenvector_centrality', 'clustering_coefficient', 'modularity', 'rich_club_coefficient_new', 'participation_coefficient_new', 'connected_components', 'minimum_spanning_tree']:
        # 比较 s2
        t_stat, p_value = compare_analysis(metrics, 'sham', 's2', condition, 's2', graph_property)
        print(f"{condition} S2 vs Sham S2 - {graph_property}: T-statistic: {t_stat}, P-value: {p_value}")


In [ ]:
print("S_3:")
for condition in ['tacs6', 'tacs10', 'tdcs']:
    for graph_property in ['degree_centrality', 'betweenness_centrality', 'eigenvector_centrality', 'clustering_coefficient', 'modularity', 'rich_club_coefficient_new', 'participation_coefficient_new', 'connected_components', 'minimum_spanning_tree']:        
        # 比较 s3
        t_stat, p_value = compare_analysis(metrics, 'sham', 's3', condition, 's3', graph_property)
        print(f"{condition} S3 vs Sham S3 - {graph_property}: T-statistic: {t_stat}, P-value: {p_value}")

In [ ]:
# 4. 比较真实刺激组在s1,s2,s3的差异
"""print("Real Stimulation Self Comparison:")
for condition in ['tacs6', 'tacs10', 'tdcs']:
    for graph_property in ['degree_centrality', 'betweenness_centrality', 'eigenvector_centrality', 'clustering_coefficient', 'modularity', 'connected_components', 'minimum_spanning_tree']:
        # 比较 s1 和 s2
        t_stat, p_value = compare_analysis(metrics, condition, 's1', condition, 's2', graph_property, paired=True)
        print(f"{condition} S1 vs S2 - {graph_property}: T-statistic: {t_stat}, P-value: {p_value}")
        
        # 比较 s1 和 s3
        t_stat, p_value = compare_analysis(metrics, condition, 's1', condition, 's3', graph_property, paired=True)
        print(f"{condition} S1 vs S3 - {graph_property}: T-statistic: {t_stat}, P-value: {p_value}")"""